<a href="https://colab.research.google.com/github/Sarthakpunj/Rag-using-vector_db-and-huugingface/blob/main/RAG_FROM_HUGGING_FACE_AND_VECTOR_DB.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install llama-index-llms-huggingface

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.7/7.7 MB 51.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 49.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 50.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 34.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 6.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 12.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 89.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 4.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 129.

In [2]:
!pip install llama-index

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.8/40.8 kB 3.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 265.9/265.9 kB 9.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 303.4/303.4 kB 22.6 MB/s eta 0:00:00


In [4]:
from llama_index.core import VectorStoreIndex,SimpleDirectoryReader
from llama_index.llms.huggingface import HuggingFaceLLM

In [6]:
!mkdir data #create a data directory

mkdir: cannot create directory ‘data’: File exists


In [7]:
#load document
documents = SimpleDirectoryReader('./data/').load_data()

In [9]:
print(documents)

[Document(id_='b2e2a6f2-310f-4151-a012-3bbf39cbe9ae', embedding=None, metadata={'page_label': '1', 'file_name': 'kb-2.pdf', 'file_path': '/content/data/kb-2.pdf', 'file_type': 'application/pdf', 'file_size': 144867, 'creation_date': '2025-05-22', 'last_modified_date': '2025-05-22'}, excluded_embed_metadata_keys=['file_name', 'file_type', 'file_size', 'creation_date', 'last_modified_date', 'last_accessed_date'], excluded_llm_metadata_keys=['file_name', 'file_type', 'file_size', 'creation_date', 'last_modified_date', 'last_accessed_date'], relationships={}, metadata_template='{key}: {value}', metadata_separator='\n', text_resource=MediaResource(embeddings=None, data=None, text='Q: What is Closify? A: Closify is an AI-powered CRM platform that simplifies and automates the entire sales and support lifecycle for businesses. It centralizes customer data, communications (like WhatsApp, Gmail, and Teams), and tasks in one place so teams can focus on relationships instead of manual processes. C

In [10]:
from llama_index.core import PromptTemplate #role = assitant(llm), user(us), system(developer env)
system_prompt = """<|SYSTEM|># You are a Q&A assistant. Your goal is to answer questions as accurately as possible based on the instructions and context provided.
"""

# This will wrap the default prompts that are internal to llama-index
query_wrapper_prompt = PromptTemplate("<|USER|>{query_str}<|ASSISTANT|>")


In [13]:
import torch

llm = HuggingFaceLLM(
    context_window=4096,
    max_new_tokens=256,
    generate_kwargs={"temperature": 0.7, "do_sample": False},
    system_prompt=system_prompt,
    query_wrapper_prompt=query_wrapper_prompt,
    tokenizer_name = "TinyLlama/TinyLlama-1.1B-Chat-v1.0",
    model_name = "TinyLlama/TinyLlama-1.1B-Chat-v1.0",
    device_map="auto",
    stopping_ids=[50278, 50279, 50277, 1, 0],
    tokenizer_kwargs={"max_length": 4096},
    # uncomment this if using CUDA to reduce memory usage
    model_kwargs={"torch_dtype": torch.float16}
)


config.json:   0%|          | 0.00/608 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/2.20G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/1.29k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.84M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/551 [00:00<?, ?B/s]

In [14]:
#embedding
!pip install llama-index-embeddings-huggingface
!pip install llama-index-embeddings-instructor

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 171.5/171.5 kB 4.6 MB/s eta 0:00:00
  Attempting uninstall: sentence-transformers
    Found existing installation: sentence-transformers 4.1.0
    Uninstalling sentence-transformers-4.1.0:
      Successfully uninstalled sentence-transformers-4.1.0


In [15]:
from llama_index.embeddings.huggingface import HuggingFaceEmbedding
embed_model = HuggingFaceEmbedding(model_name="sentence-transformers/all-mpnet-base-v2")

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.4k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/571 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/363 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [17]:
from llama_index.core import vector_stores, ServiceContext

from llama_index.core import Settings

# Configure the settings directly
Settings.llm = llm
Settings.embed_model = embed_model
Settings.chunk_size = 1024

In [18]:
index = VectorStoreIndex.from_documents(documents)

In [19]:
query= index.as_query_engine()

In [20]:
query.query("what is closify?")

/usr/local/lib/python3.11/dist-packages/transformers/generation/configuration_utils.py:631: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.7` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(


Response(response='\nClosify is an AI-powered CRM platform that simplifies and automates the entire sales and support lifecycle for businesses. It centralizes customer data, communications (like WhatsApp, Gmail, and Teams), and tasks in one place so teams can focus on relationships instead of manual processes. It uses advanced in-house AI models to analyze your data and provide insights, for example, it can sift through customer interactions and sales activity to surface patterns that humans might miss. In short, Closify helps sales and support teams manage leads, automate workflows, and close deals more effectively using AI-driven automation.', source_nodes=[NodeWithScore(node=TextNode(id_='0c9fd901-4717-4164-a048-04f0a24f0185', embedding=None, metadata={'page_label': '1', 'file_name': 'kb-2.pdf', 'file_path': '/content/data/kb-2.pdf', 'file_type': 'application/pdf', 'file_size': 144867, 'creation_date': '2025-05-22', 'last_modified_date': '2025-05-22'}, excluded_embed_metadata_keys=[